In [1]:
import matplotlib.pyplot as plt
import numpy
import pandas

import seaborn
seaborn.set_context('talk')

In [2]:
import plotly
plotly.__version__
# Make sure it's 4.14.3

'4.14.3'

In [8]:
melb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
melb_df[:]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,Wheelers Hill,12 Strada Cr,4,h,1245000.0,S,Barry,26/08/2017,16.7,3150.0,...,2.0,2.0,652.0,NaN,1981.0,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392.0
13576,Williamstown,77 Merrett Dr,3,h,1031000.0,SP,Williams,26/08/2017,6.8,3016.0,...,2.0,2.0,333.0,133.0,1995.0,NaN,-37.85927,144.87904,Western Metropolitan,6380.0
13577,Williamstown,83 Power St,3,h,1170000.0,S,Raine,26/08/2017,6.8,3016.0,...,2.0,4.0,436.0,NaN,1997.0,NaN,-37.85274,144.88738,Western Metropolitan,6380.0
13578,Williamstown,96 Verdon St,4,h,2500000.0,PI,Sweeney,26/08/2017,6.8,3016.0,...,1.0,5.0,866.0,157.0,1920.0,NaN,-37.85908,144.89299,Western Metropolitan,6380.0


In [4]:
melb_df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [5]:
def remove_outlier(df_in, col_name, iqr_factor):
    """
  IQR outliers removal (https://machinelearningmastery.com/how-to-use-statistics-to-identify-outliers-in-data/)
  accept a dataframe, remove outliers, return cleaned data in a new dataframe
  see http://www.itl.nist.gov/div898/handbook/prc/section1/prc16.htm
  """
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3 - q1  # interquartile range
    fence_low = q1 - iqr_factor * iqr
    fence_high = q3 + iqr_factor * iqr
    df_inliers = df_in.loc[
        (df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)
    ]
    df_outliers = df_in.loc[
        (df_in[col_name] < fence_low) | (df_in[col_name] > fence_high)
    ]
    return df_inliers, df_outliers

In [6]:
iqr_factor = 2.5
df_main_inliers, df_main_outliers = remove_outlier(melb_df, "Price", iqr_factor)

display(len(df_main_inliers) / len(melb_df))

0.9846833578792342

In [13]:
cols = df_main_inliers[melb_df == 0].count(axis=0)
cols[cols > 0]
#deteccion de valores nulos en las columnas

Distance           6
Bedroom2          16
Bathroom          34
Car             1021
Landsize        1937
BuildingArea      16
dtype: int64

In [9]:
df_main_inliers_copy=df_main_inliers.copy()
df_main_inliers_copy=df_main_inliers_copy.drop(['Bedroom2'], axis=1)
#se retira la variable bedroom2
df_main_inliers_copy.sample(10,random_state=123)


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
8930,Reservoir,16 Muriel Ct,3,h,825000.0,S,RW,1/07/2017,12.0,3073.0,2.0,5.0,537.0,NaN,NaN,Darebin,-37.70047,145.02881,Northern Metropolitan,21650.0
3350,Ivanhoe,9/88 Marshall St,2,u,480000.0,VB,Miles,3/09/2016,7.9,3079.0,1.0,1.0,0.0,NaN,NaN,Banyule,-37.77000,145.04720,Eastern Metropolitan,5549.0
9907,Hampton,1 Grout St,3,h,1325000.0,VB,RT,24/06/2017,13.8,3188.0,1.0,2.0,400.0,NaN,NaN,Unavailable,-37.93434,145.00816,Southern Metropolitan,5454.0
4913,Preston,16 Opal St,2,h,600000.0,S,Nelson,10/09/2016,8.8,3072.0,1.0,2.0,220.0,106.0,2005.0,Darebin,-37.74420,145.01680,Northern Metropolitan,14577.0
2670,Footscray,8 Jerrold St,2,h,752000.0,SP,Jas,7/05/2016,6.4,3011.0,1.0,0.0,215.0,98.0,1910.0,Maribyrnong,-37.79220,144.89100,Western Metropolitan,7570.0
2107,Collingwood,182 Keele St,2,h,670000.0,S,Jellis,18/03/2017,1.6,3066.0,1.0,0.0,118.0,NaN,NaN,Yarra,-37.79760,144.99300,Northern Metropolitan,4553.0
2361,Essendon,65 Napier St,3,h,1467000.0,S,Greg,7/11/2016,8.0,3040.0,2.0,2.0,188.0,171.0,1900.0,Moonee Valley,-37.75320,144.91780,Western Metropolitan,9264.0
2109,Collingwood,5/22 Stanley St,2,u,515000.0,S,Peter,18/03/2017,1.6,3066.0,1.0,1.0,816.0,NaN,NaN,Yarra,-37.80200,144.98480,Northern Metropolitan,4553.0
1971,Coburg,9 Vincent St,4,h,1155000.0,S,Nelson,18/03/2017,7.8,3058.0,1.0,2.0,370.0,NaN,NaN,Moreland,-37.74310,144.95600,Northern Metropolitan,11204.0
2640,Flemington,12/132 Princes St,2,u,556000.0,S,Woodards,16/07/2016,4.4,3031.0,2.0,1.0,0.0,79.0,2009.0,Moonee Valley,-37.78400,144.93560,Northern Metropolitan,3593.0


In [14]:
df_filtrado = df_main_inliers[['Rooms','Type','Price','Suburb','Date','Distance','Postcode',
                       'Bathroom','Car','Landsize','BuildingArea','YearBuilt','Regionname','Propertycount' ]]
df_filtrado
#saco sellerG al no importar quien vendio y method al entender que no importa como fue que se vendio.
#agrego suburb, regionname, propertycount
#creo que hay muchas variables de ubicacion. Habria q ver cual sirve y las otras descartar.

,Rooms,Type,Price,Suburb,Date,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Regionname,Propertycount
0,2,h,1480000.0,Abbotsford,3/12/2016,2.5,3067.0,1.0,1.0,202.0,NaN,NaN,Northern Metropolitan,4019.0
1,2,h,1035000.0,Abbotsford,4/02/2016,2.5,3067.0,1.0,0.0,156.0,79.0,1900.0,Northern Metropolitan,4019.0
2,3,h,1465000.0,Abbotsford,4/03/2017,2.5,3067.0,2.0,0.0,134.0,150.0,1900.0,Northern Metropolitan,4019.0
3,3,h,850000.0,Abbotsford,4/03/2017,2.5,3067.0,2.0,1.0,94.0,NaN,NaN,Northern Metropolitan,4019.0
4,4,h,1600000.0,Abbotsford,4/06/2016,2.5,3067.0,1.0,2.0,120.0,142.0,2014.0,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,4,h,1245000.0,Wheelers Hill,26/08/2017,16.7,3150.0,2.0,2.0,652.0,NaN,1981.0,South-Eastern Metropolitan,7392.0
13576,3,h,1031000.0,Williamstown,26/08/2017,6.8,3016.0,2.0,2.0,333.0,133.0,1995.0,Western Metropolitan,6380.0
13577,3,h,1170000.0,Williamstown,26/08/2017,6.8,3016.0,2.0,4.0,436.0,NaN,1997.0,Western Metropolitan,6380.0
13578,4,h,2500000.0,Williamstown,26/08/2017,6.8,3016.0,1.0,5.0,866.0,157.0,1920.0,Western Metropolitan,6380.0


In [51]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13372 entries, 0 to 13579
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rooms          13372 non-null  int64  
 1   Type           13372 non-null  object 
 2   Price          13372 non-null  float64
 3   Suburb         13372 non-null  object 
 4   Date           13372 non-null  object 
 5   Distance       13372 non-null  float64
 6   Postcode       13372 non-null  float64
 7   Bathroom       13372 non-null  float64
 8   Car            13310 non-null  float64
 9   Landsize       13372 non-null  float64
 10  BuildingArea   6998 non-null   float64
 11  YearBuilt      8064 non-null   float64
 12  Regionname     13372 non-null  object 
 13  Propertycount  13372 non-null  float64
dtypes: float64(9), int64(1), object(4)
memory usage: 2.2+ MB


In [21]:
df_filtrado.Type.value_counts()


h    9244
u    3016
t    1112
Name: Type, dtype: int64

In [22]:
df_filtrado.Suburb.value_counts()

Reservoir         359
Richmond          258
Bentleigh East    249
Preston           239
Brunswick         222
                 ... 
Healesville         1
Montrose            1
Rockbank            1
Wallan              1
New Gisborne        1
Name: Suburb, Length: 314, dtype: int64

In [67]:
#agrupo en otros para disminuir cantidad de categorias
df_filtrado[['Suburb', 'Postcode']].groupby( 'Postcode').describe()

Suburb                            
          count unique             top freq
Postcode                                   
3000.0       46      1       Melbourne   46
3002.0       19      1  East Melbourne   19
3003.0       31      1  West Melbourne   31
3006.0       41      1       Southbank   41
3008.0        3      1       Docklands    3
...         ...    ...             ...  ...
3809.0        1      1         Officer    1
3810.0        3      1        Pakenham    3
3910.0        6      1      Langwarrin    6
3976.0        4      1    Hampton Park    4
3977.0        8      4      Cranbourne    3

[198 rows x 4 columns]

In [73]:
df_filtrado.Suburb.value_counts().tail(50)

The Basin             4
Croydon Hills         4
Bonbeach              4
Campbellfield         4
Coolaroo              4
Springvale South      3
Seabrook              3
St Helena             3
Cranbourne            3
Burnside Heights      3
Ardeer                3
Notting Hill          3
Docklands             3
Pakenham              3
Williams Landing      2
Chirnside Park        2
Hurstbridge           2
Keilor Lodge          2
Cranbourne North      2
Beaconsfield          2
Skye                  2
Diggers Rest          2
Waterways             2
Burnside              2
Mount Evelyn          2
North Warrandyte      2
Silvan                2
Deepdene              2
Croydon South         1
Bacchus Marsh         1
Attwood               1
Bullengarook          1
Upwey                 1
Emerald               1
Monbulk               1
Whittlesea            1
Plumpton              1
Kooyong               1
Brookfield            1
Beaconsfield Upper    1
Wonga Park            1
Sandhurst       

In [43]:
df_filtrado.Regionname.value_counts()

Southern Metropolitan         4510
Northern Metropolitan         3877
Western Metropolitan          2944
Eastern Metropolitan          1466
South-Eastern Metropolitan     449
Eastern Victoria                53
Northern Victoria               41
Western Victoria                32
Name: Regionname, dtype: int64

In [49]:
df_filtrado.Postcode.value_counts()

3073.0    359
3020.0    306
3121.0    290
3040.0    289
3046.0    284
         ... 
3775.0      1
3777.0      1
3809.0      1
3808.0      1
3793.0      1
Name: Postcode, Length: 198, dtype: int64

In [55]:
df_filtrado['date'] = pandas.to_datetime(df_filtrado.Date)
#se convierte a formato tiempo para poder graficar

<ipython-input-55-80ed8fd6fdde>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['date'] = pandas.to_datetime(df_filtrado.Date)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 13372 entries, 0 to 13579
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Rooms          13372 non-null  int64         
 1   Type           13372 non-null  object        
 2   Price          13372 non-null  float64       
 3   Suburb         13372 non-null  object        
 4   Date           13372 non-null  object        
 5   Distance       13372 non-null  float64       
 6   Postcode       13372 non-null  float64       
 7   Bathroom       13372 non-null  float64       
 8   Car            13310 non-null  float64       
 9   Landsize       13372 non-null  float64       
 10  BuildingArea   6998 non-null   float64       
 11  YearBuilt      8064 non-null   float64       
 12  Regionname     13372 non-null  object        
 13  Propertycount  13372 non-null  float64       
 14  date           13372 non-null  datetime64[ns]
dtypes: datetime64[ns](1